# Laboratorio 7. Predicción de datos con AutoGluon



Se estará usando una base de datos de diabetes, con la siguiente distribuición 
Columnas en el archivo diabetes.csv
* Pregnancies:  Número de veces que ha estado embarazada
* GlucosePlasma:  Concentración de glucosa
* BloodPressure:  Presión de sangre diastólica (mm Hg)
* Skin:  Grosor del pliegue cutáneo del Triscep (mm)
* Insulin:  Suero de insulina 2-Horas (mu U/ml)
* BMI:  Indice de masa corporal (peso en Kg/(estatura en mts)2)
* DiabetesPedigreeFunction:  Función de pedigree de diabetes
* Age:  Edad en años
* Outcome:  Diabetes ó no diabetes (0 ó 1)

## Análisis Exploratorio 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

# Configuración para visualizaciones
plt.style.use('default')
sns.set_palette("husl")

# Cargar datos
df = pd.read_csv("./diabetes.csv")

print("=" * 60)
print("ANÁLISIS EXPLORATORIO DE DATOS - DATASET DIABETES")
print("=" * 60)

# 1. INFORMACIÓN BÁSICA DEL DATASET
print("\n1. INFORMACIÓN BÁSICA DEL DATASET")
print("-" * 40)
print(f"Forma del dataset: {df.shape}")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

print("\nTipos de datos:")
print(df.dtypes)

print("\nPrimeras 5 filas:")
print(df.head())

# 2. ESTADÍSTICAS DESCRIPTIVAS
print("\n\n2. ESTADÍSTICAS DESCRIPTIVAS")
print("-" * 40)
print(df.describe())

# 3. VERIFICACIÓN DE VALORES NULOS
print("\n\n3. VALORES NULOS")
print("-" * 40)
valores_nulos = df.isnull().sum()
print("Valores nulos por columna:")
print(valores_nulos)

if valores_nulos.sum() == 0:
    print("✅ No hay valores nulos en el dataset")
else:
    print("⚠️ Se encontraron valores nulos")

# 4. VERIFICACIÓN DE VALORES CERO (POSIBLES VALORES ATÍPICOS)
print("\n\n4. VALORES CERO (POSIBLES ATÍPICOS)")
print("-" * 40)
# Columnas donde 0 podría ser un valor atípico
columnas_cero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in columnas_cero:
    ceros = (df[col] == 0).sum()
    print(f"{col}: {ceros} valores cero ({ceros/len(df)*100:.1f}%)")

# 5. ANÁLISIS DE BALANCE DE CLASES
print("\n\n5. BALANCE DE CLASES (OUTCOME)")
print("-" * 40)
conteo_clases = df['Outcome'].value_counts()
print("Distribución de clases:")
print(conteo_clases)

porcentajes = df['Outcome'].value_counts(normalize=True) * 100
print("\nPorcentajes:")
for clase, porcentaje in porcentajes.items():
    etiqueta = "No Diabetes" if clase == 0 else "Diabetes"
    print(f"{etiqueta} (Clase {clase}): {porcentaje:.1f}%")

# 6. VISUALIZACIONES
print("\n\n6. GENERANDO VISUALIZACIONES...")
print("-" * 40)

# Configurar el estilo de las gráficas
plt.rcParams['figure.figsize'] = (15, 12)

# 6.1 Histogramas
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for i, col in enumerate(df.columns):
    if col != 'Outcome':
        axes[i].hist(df[col], bins=20, alpha=0.7, edgecolor='black')
        axes[i].set_title(f'Distribución de {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frecuencia')

# Gráfico de barras para Outcome
axes[8].bar(conteo_clases.index, conteo_clases.values, color=['skyblue', 'lightcoral'])
axes[8].set_title('Distribución de Outcome')
axes[8].set_xlabel('Outcome')
axes[8].set_ylabel('Frecuencia')
axes[8].set_xticks([0, 1])
axes[8].set_xticklabels(['No Diabetes', 'Diabetes'])

plt.tight_layout()
plt.suptitle('Histogramas de Variables', fontsize=16, y=1.02)
plt.show()

# 6.2 Boxplots
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for i, col in enumerate(df.columns[:-1]):  # Excluir 'Outcome'
    axes[i].boxplot(df[col])
    axes[i].set_title(f'Boxplot de {col}')
    axes[i].set_ylabel(col)

plt.tight_layout()
plt.suptitle('Boxplots de Variables', fontsize=16, y=1.02)
plt.show()

# 6.3 Matriz de correlación
print("Generando matriz de correlación...")
correlacion = df.corr()

plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlacion, dtype=bool))
sns.heatmap(correlacion, 
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            mask=mask,
            square=True,
            fmt='.2f',
            cbar_kws={"shrink": .8})
plt.title('Matriz de Correlación', fontsize=16)
plt.tight_layout()
plt.show()

# 7. RESUMEN DE CORRELACIONES IMPORTANTES
print("\n\n7. CORRELACIONES MÁS IMPORTANTES CON OUTCOME")
print("-" * 40)
correlacion_outcome = correlacion['Outcome'].abs().sort_values(ascending=False)[1:]  # Excluir la correlación consigo mismo
print("Variables más correlacionadas con Outcome:")
for variable, corr in correlacion_outcome.items():
    print(f"{variable}: {corr:.3f}")

# 8. DETECCIÓN DE OUTLIERS (MÉTODO IQR)
print("\n\n8. DETECCIÓN DE OUTLIERS (MÉTODO IQR)")
print("-" * 40)

def detectar_outliers(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    outliers = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)]
    return len(outliers)

for col in df.columns[:-1]:  # Excluir 'Outcome'
    num_outliers = detectar_outliers(df, col)
    print(f"{col}: {num_outliers} outliers")

print("\n" + "=" * 60)
print("ANÁLISIS COMPLETADO")
print("=" * 60)